# Skorch introduction

(borrowed from [skorch](https://skorch.readthedocs.io) documentation)

*`skorch`* is designed to maximize interoperability between `sklearn` and `pytorch`. The aim is to keep 99% of the flexibility of `pytorch` while being able to leverage most features of `sklearn`. Below, we show the basic usage of `skorch` and how it can be combined with `sklearn`.


In [0]:
! [ ! -z "$COLAB_GPU" ] && pip install skorch scikit-learn==0.20.3

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(0);


## Training a classifier and making predictions

### A toy binary classification task

We load a toy classification task from `sklearn`.

In [0]:
import numpy as np
from sklearn.datasets import make_classification

In [0]:
X, y = make_classification(1000, 20, n_informative=10, n_classes=2, random_state=0)
X = X.astype(np.float32)

In [0]:
X.shape, y.shape, y.mean()

### Definition of the `pytorch` classification `module`

We define a vanilla neural network with two hidden layers. The output layer should have 2 output units since there are two classes. In addition, it should have a softmax nonlinearity, because later, when calling `predict_proba`, the output from the `forward` call will be used.

In [0]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

### Defining and training the neural net classifier

We use `NeuralNetClassifier` because we're dealing with a classifcation task. The first argument should be the `pytorch module`. As additional arguments, we pass the number of epochs and the learning rate (`lr`), but those are optional.

*Note*: To use the CUDA backend, pass `device='cuda'` as an additional argument.

In [0]:
from skorch import NeuralNetClassifier

In [0]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device='cuda',  # comment this to train with CPU
)

As in `sklearn`, we call `fit` passing the input data `X` and the targets `y`. By default, `NeuralNetClassifier` makes a `StratifiedKFold` split on the data (80/20) to track the validation loss. This is shown, as well as the train loss and the accuracy on the validation set.

In [0]:
net.fit(X, y)

Also, as in `sklearn`, you may call `predict` or `predict_proba` on the fitted model.

### Making predictions, classification

In [0]:
y_pred = net.predict(X[:5])
y_pred

In [0]:
y_proba = net.predict_proba(X[:5])
y_proba

## Usage with sklearn `GridSearchCV`

### Special prefixes

The `NeuralNet` class allows to directly access parameters of the `pytorch module` by using the `module__` prefix. So e.g. if you defined the `module` to have a `num_units` parameter, you can set it via the `module__num_units` argument. This is exactly the same logic that allows to access estimator parameters in `sklearn Pipeline`s and `FeatureUnion`s.

This feature is useful in several ways. For one, it allows to set those parameters in the model definition. Furthermore, it allows you to set parameters in an `sklearn GridSearchCV` as shown below.

In addition to the parameters prefixed by `module__`, you may access a couple of other attributes, such as those of the optimizer by using the `optimizer__` prefix (again, see below). All those special prefixes are stored in the `prefixes_` attribute:

In [0]:
print(', '.join(net.prefixes_))

### Performing a grid search

Below we show how to perform a grid search over the learning rate (`lr`), the module's number of hidden units (`module__num_units`), the module's dropout rate (`module__dropout`), and whether the SGD optimizer should use Nesterov momentum or not (`optimizer__nesterov`).

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    verbose=0,
    optimizer__momentum=0.9,
)

In [0]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': <YOUR CODE>, # range from 10 to 50
    'module__dropout': <YOUR CODE>, # range from 0.1 to 0.3
    'optimizer__nesterov': [False, True],
}


In [0]:
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', 
                  verbose=1, n_jobs=2)

In [0]:
gs.fit(X, y);

In [0]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [0]:
report(gs.cv_results_)

Other skorch capabilities:

- [Saving and loading models](https://skorch.readthedocs.io/en/stable/user/save_load.html)
- [Accessing history of network training](https://skorch.readthedocs.io/en/stable/user/history.html)
- [Enhance training by callbacks](https://skorch.readthedocs.io/en/stable/user/callbacks.html)

also link to examples: https://skorch.readthedocs.io/en/stable/user/tutorials.html